# 缺失值

## 缺失值的定义
**缺失值（Missing Value）**是指数据集中某些观测值或特征值未被记录或无法获取的情况，通常以特殊符号表示，如：
- **数值型数据**：NaN（Not a Number）、NA、None、NULL
- **分类数据**："Unknown"、"N/A"、空字符串（""）

## 缺失值的产生原因
缺失值可能由多种因素导致，主要包括：
- **数据采集问题**：传感器故障、人工录入遗漏、网络传输中断等。
- **数据存储问题**：数据库损坏、存储格式转换错误。
- **数据逻辑问题**：某些特征不适用于特定样本（如“怀孕次数”对男性无意义）。

## 缺失值的类型
| 缺失机制 | 英文全称                      | 定义                                                                 | 示例                                                                 |
|----------|-----------------------------|----------------------------------------------------------------------|----------------------------------------------------------------------|
| MCAR     | Missing Completely At Random | 缺失完全随机，与任何变量无关（观测数据与缺失数据无差异）                  | 实验设备随机故障导致部分数据丢失                                        |
| MAR      | Missing At Random           | 缺失与已观测变量相关，但与缺失值本身无关（可建模预测）                    | 收入数据缺失可能与性别相关（如女性更不愿透露收入），但同性别内缺失是随机的 |
| MNAR     | Missing Not At Random       | 缺失与缺失值本身相关（存在系统性偏差，难以直接建模）                      | 高收入人群更可能隐藏收入，导致收入缺失值与真实收入相关                  |

## 缺失值的影响
- **统计分析偏差**：若直接删除缺失数据（尤其是MNAR），可能导致结果偏离真实分布。
- **模型性能下降**：大多数机器学习算法（如线性回归、SVM）无法直接处理缺失值，需预处理。
- **计算错误**：部分数学运算（如求和、均值）遇到缺失值可能返回NaN或报错。

# 泰坦尼克数据集

## 泰坦尼克数据集

**titanic 数据集**包含了 891 条乘客记录，每条记录描述了乘客的个人信息及其在 Titanic 号上的生存情况
- survived：生存情况（0 = 未生还, 1 = 生还）
- pclass：乘客舱等级（1 = 一等舱, 2 = 二等舱, 3 = 三等舱）
- sex：乘客性别（male = 男性, female = 女性）
- age：乘客年龄（float，部分缺失值）
- sibsp：乘客在船上的兄弟姐妹或配偶数量（整数）
- parch：乘客在船上的父母或子女数量（整数）
- fare：乘客支付的票价（float）
- embarked：乘客登船港口（C = 瑟堡, Q = 皇后镇, S = 南安普敦）
- class：舱等级（文本版本的 pclass，First = 头等舱, Second = 二等舱, Third = 三等舱）
- who：乘客身份（man = 成年男性, woman = 成年女性, child = 儿童）
- adult_male：是否为成年男性（True = 是, False = 否）
- deck：乘客所在的甲板（A, B, C, D, E, F, G，部分缺失值较多）
- embark_town：乘客登船的城市（Cherbourg = 瑟堡, Queenstown = 皇后镇, Southampton = 南安普敦）
- alone：是否独自旅行（True = 独自旅行，False = 有亲属同行，对应 sibsp + parch == 0）

In [17]:
# 导入必要的库
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, confusion_matrix, 
                           classification_report, roc_curve, auc)
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# 加载seaborn内置的泰坦尼克数据集
df = sns.load_dataset('titanic')

df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


## 缺失值检测

In [18]:
# 缺失值统计
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [19]:
# 查看缺失比例
df.isnull().mean()

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64

# 缺失值处理方法

## 删除法

### `df.dropna()`
- `axis`: 数据删除方向
  - `0`: 删除行
  - `1`: 删除列
- `how`: 控制删除条件
  - `any`: 只要有一个缺失值就删除该行/列（默认）
  - `all`: 所有值都缺失才删除该行/列
- `subset`: 指定需要处理缺失值的行/列。默认为全体数据
- `inplace`: 是否原地修改数据
  - `True`: 直接修改原 DataFrame
  - `False`: 返回一个新对象
- `ignore_index`: 删除行后是否重置索引

### 删除含缺失值的样本（行）
- 适用：当缺失值数量较少，对样本数量影响不大时。
- 缺点：可能导致信息损失，特别是在缺失值多的时候。

In [20]:
df.dropna(axis=0)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,First,woman,False,C,Southampton,yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,1,1,female,47.0,1,1,52.5542,S,First,woman,False,D,Southampton,yes,False
872,0,1,male,33.0,0,0,5.0000,S,First,man,True,B,Southampton,no,True
879,1,1,female,56.0,0,1,83.1583,C,First,woman,False,C,Cherbourg,yes,False
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True


### 删除含缺失值的变量（列）
- 适用：某一列缺失值占比很高，无法通过其他手段合理填补时。
- 缺点：可能丢失重要特征。

In [21]:
df.dropna(axis=1)

,survived,pclass,sex,sibsp,parch,fare,class,who,adult_male,alive,alone
0,0,3,male,1,0,7.2500,Third,man,True,no,False
1,1,1,female,1,0,71.2833,First,woman,False,yes,False
2,1,3,female,0,0,7.9250,Third,woman,False,yes,True
3,1,1,female,1,0,53.1000,First,woman,False,yes,False
4,0,3,male,0,0,8.0500,Third,man,True,no,True
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,13.0000,Second,man,True,no,True
887,1,1,female,0,0,30.0000,First,woman,False,yes,True
888,0,3,female,1,2,23.4500,Third,woman,False,no,False
889,1,1,male,0,0,30.0000,First,man,True,yes,True


## 填充法

### `df.fillna()`
- `value`: 用于指定填充缺失值的具体值
  - 单个数值（如 0, -1, “missing”）
  - 字典（为每列指定不同的填充值）
- `method`: 填充策略
  - `ffill`: 前向填充（使用上一个非缺失值）
  - `bfill`: 后向填充（使用下一个非缺失值）
- `value` 和 `method` 不能同时使用
- `axis`: 填充方向
  - axis=0：纵向填充（按列）
  - axis=1：横向填充（按行）
- `inplace`: 是否原地修改数据
  - `True`: 直接修改原 DataFrame
  - `False`: 返回一个新对象

### 数值型变量填充

#### 均值填充

**均值填充（mean）**：适合数据分布近似正态。

In [22]:
# 均值填充
df_mean = df.copy()
df_mean["age"].fillna(df_mean["age"].mean(), inplace=True)
df_mean

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.000000,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,29.699118,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.000000,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


#### 中位数填充

**中位数填充（median）**：适合偏态分布数据，抗离群值能力强。

In [23]:
# 中位数填充
df_median = df.copy()
df_median["age"].fillna(df_median["age"].median(), inplace=True)
df_median

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,28.0,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


#### 众数填充

**众数填充（mode）**：适合离散型数据。

In [26]:
# 众数填充（分类变量）
df_mode = df.copy()
df_mode["deck"].fillna(df_mode["deck"].mode()[0], inplace=True)
df_mode

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,C,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,C,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,C,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,C,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,C,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


### 分组填充（Group-wise Imputation）

**分组填充**: 对某变量按组（如性别）计算均值/中位数/众数填充，特别适合组内差异显著时。

In [27]:
# 按 sex 分组填充 age
df_grouped = df.copy()
df_grouped["age"] = df_grouped.groupby("sex")["age"].transform(lambda x: x.fillna(x.mean()))
df_grouped

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.000000,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,27.915709,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.000000,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


### 前向/后向填充（用于时间序列数据）

适用于时间序列数据，根据时间顺序填补缺失。

#### 前向填充

**前向填充（ffill）**：用前一个值填充。

In [30]:
# titanic非时间序列数据集，仅供演示使用
df_ffill = df.copy()
df_ffill["deck"].fillna(method="ffill", inplace=True)
df_ffill

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,C,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,C,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,C,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,B,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


#### 后向填充

**后向填充（bfill）**：用后一个值填充。

In [32]:
# titanic非时间序列数据集，仅供演示使用
df_bfill = df.copy()
df_bfill["deck"].fillna(method="bfill", inplace=True)
df_bfill

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,C,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,C,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,E,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,B,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,C,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


### 固定值填充

用 0、-1、"未知" 等填充，适用于分类变量，表示“缺失”本身。

In [33]:
# 创建数据副本，避免修改原始 DataFrame
df_filled = df.copy()

# 使用固定值 -1 填充 age 中的缺失值
df_filled["age"].fillna(value=-1, inplace=True)

df_filled

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,-1.0,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


## 插值法（Interpolation）

使用插值技术估计中间缺失值，如线性、多项式、样条等。

### `df.interpolate()`: 
- 根据已有数据点，通过插值方法填充 NaN 值，常用于数值连续性强、时间序列或逻辑顺序明确的数据。
- `method`: 插值方式
  - `linear`: 线性插值（默认）
      - 利用缺失值前后的两个非空点，按直线斜率插值。
      - 数学公式: $$y = y_1 + (x - x_1)\frac{y_2 - y_1}{x_2 - x_1}$$
      - 快速、简单、稳定。对于趋势平稳或线性变化的数据很有效。
  - `polynomial`: 多项式插值（需指定 多项式次数order）
      - 使用一个**多项式函数（如二次或三次函数）**拟合数据并估算中间缺失值。 
      - 数学公式: $$f(x) = a_0 + a_1x + a_2x^2 + \dots + a_nx^n$$
      - 能够模拟非线性趋势。但是对噪声敏感，容易过拟合。
  - `spline`: 样条插值（需安装 scipy）
      - 将数据划分为多个区间，每个区间使用低阶多项式拟合，同时保证边界光滑（连续导数）
      - 比多项式更平滑、鲁棒。 更适合光滑曲线型数据（如股价、温度）。
  - `time`: 基于时间索引插值
      -  把 DataFrame.index 当作时间点，用这些时间点的间隔作为插值的参考。
      - 适用于时间序列（例如股价、气温等）。要求索引是 datetime 类型。
- `axis`: 控制插值方向
  - axis=0：按列插值（常用）
  - axis=1：按行插值
- `limit`: 填充策略
  - 最多插值多少个连续的 NaN
  - 超出限制的 NaN 不会被填补
- `limit_direction`: 控制插值的方向
  - `forward`: 仅向前插
  - `backward`: 仅向后插
  - `both`: 双向插值（适合中间缺失）
- `inplace`: 是否原地修改数据
  - `True`: 直接修改原 DataFrame
  - `False`: 返回一个新对象

In [34]:
df_interp = df.copy()
df_interp["age"] = df_interp["age"].interpolate(method="linear")
df_interp

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,22.5,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


## 预测法

### 回归填补

使用其他变量构建模型（如线性回归、逻辑回归）预测缺失值

In [35]:
# 导入线性回归模型
from sklearn.linear_model import LinearRegression

# 选择与年龄相关的特征列：age（目标），fare（船票价格），pclass（舱位等级）
df_reg = df[["age", "fare", "pclass"]].copy()

# 构建训练集：只保留 age 非缺失的样本用于训练模型
train = df_reg[df_reg["age"].notnull()]

# 构建测试集：只保留 age 缺失的样本，用于预测插补
test = df_reg[df_reg["age"].isnull()]

# 初始化线性回归模型
model = LinearRegression()

# 拟合模型，用训练集中 fare 和 pclass 预测 age
model.fit(train[["fare", "pclass"]], train["age"])

# 对测试集中的缺失 age 进行预测
predicted = model.predict(test[["fare", "pclass"]])

# 将预测结果填回原始数据中的 age 缺失位置
df_reg.loc[df["age"].isnull(), "age"] = predicted

df_reg

,age,fare,pclass
0,22.000000,7.2500,3
1,38.000000,71.2833,1
2,26.000000,7.9250,3
3,35.000000,53.1000,1
4,35.000000,8.0500,3
...,...,...,...
886,27.000000,13.0000,2
887,19.000000,30.0000,1
888,24.150209,23.4500,3
889,26.000000,30.0000,1


### K近邻填补（KNN Imputation）

通过与其他样本的“相似度”寻找最近的K个邻居，用它们的均值/众数填补

#### KNNImputer()
使用 K 近邻算法对缺失值进行填补
- 对于每一个包含缺失值的样本，从其他样本中找到最近的 K 个邻居（不考虑缺失值位置）。
- 利用这 K 个邻居在该列中的数值，进行**均值（或加权平均）**填补。

- `missing_values`: 要填补的缺失标记，默认是`np.nan
- `n_neighbors`: 邻居数量
- `weights`: 邻居加权策略
  - `uniform`：等权平均
  - `distance`：距离越近权重越大
- `metric`: 距离函数，默认是`nan_euclidean
- `keep_empty_features`: 是否保留全为缺失的特征

#### KNNImputer()的方法

| 方法名               | 是否训练相关 | 返回值/说明                                       | 适用说明 |
|----------------------|--------------|---------------------------------------------------|----------|
| `fit(X)`             | ✅ 是         | 拟合模型，无返回值                                 | 计算训练数据中各样本之间的距离，用于后续填补 |
| `transform(X)`       | ❌ 否         | 返回补全后的数据（ndarray）                        | 使用最近邻对缺失值进行填补（要求之前已 `fit`） |
| `fit_transform(X)`   | ✅ 是         | 一步完成拟合和填补，返回补全数据                   | 常用方法，对含缺失值的数据直接处理 |
| `get_params()`       | ❌ 否         | 返回构造函数中使用的参数字典                        | 查看或调试当前模型配置 |

#### KNNImputer()的代码实现

In [36]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# 简化示例：只保留数值列
df_knn = df[["age", "fare", "pclass"]].copy()

imputer = KNNImputer(n_neighbors=3)
df_knn_imputed = pd.DataFrame(imputer.fit_transform(df_knn), columns=df_knn.columns)
df_knn_imputed

,age,fare,pclass
0,22.0,7.2500,3.0
1,38.0,71.2833,1.0
2,26.0,7.9250,3.0
3,35.0,53.1000,1.0
4,35.0,8.0500,3.0
...,...,...,...
886,27.0,13.0000,2.0
887,19.0,30.0000,1.0
888,30.0,23.4500,3.0
889,26.0,30.0000,1.0


### 逐特征建模插补（Multivariate Imputation by Chained Equations, MICE）

- 对于每一个有缺失值的特征$X_j$：
    - 把其他特征$X_{-j}$作为自变量，
    - 使用 estimator（默认是线性回归）来预测$X_j$中缺失的值。
- 循环遍历所有包含缺失值的特征，重复上述过程。
- 重复多轮（默认最多 max_iter=10 次），直到变化收敛或达到最大轮数。

#### IterativeImputer()

- `estimator`: 插补时使用的预测模型，默认是线性回归。可替换为 RandomForestRegressor、BayesianRidge、KNeighborsRegressor 等
- `missing_values`: 要填补的缺失标记，默认是`np.nan
- `max_iter`: 最大迭代次数（每一轮迭代会为每一列插补一次）。默认值10
- `n_nearest_features`: 限制用于预测的特征数量，选取最近的 N 个特征（基于相关性）。默认 None，也即使用全部非目标特征。
- `initial_strategy`: 初始插补策略（第一轮迭代前，先粗略填补缺失值）
    - `mean`：均值
    - `median`：中位数
    - `most_frequent`：众数
    - `constant`：指定值（结合 fill_value）
- `fill_value`: 配合`initial_strategy='constant'`使用，设定初始填充值
- `imputation_order`: 控制插补顺序
    - `ascending`：优先填补缺失值较少的特征（推荐）
    - `descending`：优先填补缺失值最多的特征
    - `roman`：按列顺序
    - `arabic`：按倒序
    - `random`：随机顺序（需设定 random_state）
- `min_value / max_value`（默认 -inf / inf）
    - 限制插补值的范围。
    - 例如防止出现负数年龄等逻辑错误。
- `verbose`: 控制输出信息的详细程度
    - `0`：不输出
    - `1`：每轮迭代打印损失值
- `random_state`: 随机数种子
- `keep_empty_features`: 是否保留全为缺失的特征    

#### IterativeImputer()的方法

| 方法名               | 是否训练相关 | 返回值 / 说明                                   | 适用说明 |
|----------------------|--------------|--------------------------------------------------|----------|
| `fit(X)`             | ✅ 是         | 拟合模型，无返回值                               | 对缺失值建模，用于训练阶段，估计每列的预测器 |
| `transform(X)`       | ❌ 否         | 返回补全后的数据（ndarray）                      | 使用训练好的模型对缺失值进行多轮预测填补 |
| `fit_transform(X)`   | ✅ 是         | 一步完成拟合和填补，返回补全数据                | 最常用，直接对原始数据进行缺失值预测补全 |
| `get_params()`       | ❌ 否         | 返回当前实例的参数字典                           | 用于查看当前模型的参数配置 |
| `set_params(**params)` | ❌ 否       | 设置参数值并返回自身                              | 支持模型调参或在网格搜索中使用 |

#### IterativeImputer()的代码实现

In [37]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df_iter = df[["age", "fare", "pclass"]].copy()

imp = IterativeImputer(max_iter=10, random_state=0)
df_iter_imputed = pd.DataFrame(imp.fit_transform(df_iter), columns=df_iter.columns)
df_iter_imputed

,age,fare,pclass
0,22.000000,7.2500,3.0
1,38.000000,71.2833,1.0
2,26.000000,7.9250,3.0
3,35.000000,53.1000,1.0
4,35.000000,8.0500,3.0
...,...,...,...
886,27.000000,13.0000,2.0
887,19.000000,30.0000,1.0
888,24.237629,23.4500,3.0
889,26.000000,30.0000,1.0


# 总结

| 方法分类    | 方法名称                               | 方法介绍                            | 适用场景                  | 优点                 | 缺点                   |
| ------- | ---------------------------------- | ------------------------------- | --------------------- | ------------------ | -------------------- |
| **删除法** | 删除行（`dropna`）                      | 删除包含缺失值的整行数据                    | 缺失比例很小或行不重要           | 简单直接；适用于缺失少量数据的场景  | 信息损失；数据量减少           |
|         | 删除列（`dropna`）                      | 删除缺失值占比高的列                      | 缺失列几乎完全为空或不具分析价值      | 清理冗余特征；维度简化        | 可能删除重要特征；影响模型性能      |
| **填充法** | 均值填充（`fillna(mean)`）               | 用该列所有非缺失值的均值填补                  | 数值型变量、分布近似正态          | 实现简单；保留样本完整性       | 易受异常值影响；可能引入偏差       |
|         | 中位数填充                              | 用该列非缺失值的中位数进行填充                 | 数值型变量、含异常值            | 抗异常值能力强            | 忽略数据间相关性             |
|         | 众数填充                               | 用该列最频繁出现的值填补（常用于分类变量）           | 类别型变量                 | 不改变类别分布            | 对稀有类别不友好；可能增强偏差      |
|         | 分组填充                               | 按某个字段（如性别/舱位）分组后，用组内均值/中位数/众数填充 | 不同组别分布差异明显（如男女平均年龄不同） | 更合理反映组内差异；提升填补准确性  | 实现略复杂；对小样本组效果较差      |
|         | 前向填充（`ffill`）                      | 用前一行的值填充当前缺失值                   | 时间序列数据；缺失值分布较连续       | 保留局部趋势；易实现         | 连续缺失可能填不全；误差可累积      |
|         | 后向填充（`bfill`）                      | 用下一行的值填充当前缺失值                   | 同上                    | 同前向填充              | 同上                   |
|         | 固定值填充                              | 用指定值（如 `-1`、`"未知"`）填充           | 需要标识缺失信息场景（如树模型）      | 保留缺失信息；适合树模型       | 不适合线性模型；不具实际含义       |
| **插值法** | 线性插值（`interpolate('linear')`）      | 以相邻两点构成直线进行插值                   | 时间序列/数值连续变量           | 实现简单；插值自然          | 仅适用于线性趋势变量           |
|         | 多项式插值                              | 使用多项式拟合曲线进行插值（需指定阶数）            | 曲线型趋势数据               | 拟合灵活，可逼近非线性变化      | 阶数选择难；易过拟合           |
|         | 样条插值                               | 使用 B 样条或样条曲线插值                  | 曲线平滑变化的数据             | 平滑性好；适合高阶连续性数据     | 需依赖 `scipy` 等库；对噪声敏感 |
|         | 时间插值（`interpolate(method='time')`） | 基于时间索引进行插值（必须设置时间索引）            | 有时间索引的数据集             | 利用时间特性；适合金融/传感器等数据 | 时间顺序依赖性强，不适用于无时间轴数据  |
| **预测法** | 回归填补                               | 构造模型（如线性回归）预测缺失值                | 有强相关特征时（如年龄与票价/舱位）    | 保留变量间关系；预测能力强      | 需要建模；对异常值敏感          |
|         | KNN填补（`KNNImputer`）                | 根据最近 K 个样本的平均值填补缺失值             | 数值型变量、变量间相关性强         | 简单有效；保留局部结构信息      | 计算开销大；需标准化数据         |
|         | 迭代插补（`IterativeImputer`）           | 多次迭代拟合多个变量之间的关系，逐特征建模填补         | 多变量协同缺失、变量间高度相关       | 灵活强大；可用任意回归器建模     | 计算复杂；可能引入模型误差        |
